In [65]:
import pandas as pd

# CSVデータの読み込み
data = pd.read_csv("original_data/merged_data.csv")

# DataFrameの作成
df = pd.DataFrame(data)

/tmp/ipykernel_3086/1858227074.py:4: DtypeWarning: Columns (1,12) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("original_data/merged_data.csv")


In [66]:
# Horse Weightを変形
df[['Weight', 'Weight Change']] = df['Horse Weight'].str.extract(r'(\d+)\s*\(\s*([+-]?\d+)\s*\)')
df['Weight'] = df['Weight'].fillna(0)
df['Weight'] = df['Weight'].astype(int)
df['Weight Change'] = df['Weight Change'].fillna(0)
df['Weight Change'] = df['Weight Change'].astype(int)
df.drop(columns=['Horse Weight'], inplace=True)

In [67]:
# 「Sex/Age」列を「Sex」と「Age」に分ける
df['Sex'] = df['Sex/Age'].str[0].map({'牡': 0, '牝': 1, 'セ' : 0}).astype('Int64')  
df['Age'] = pd.to_numeric(df['Sex/Age'].str[1:], errors='coerce').astype('Int64') # errors='coerce'でエラーをNaNに変換
df.drop(columns=['Sex/Age'], inplace=True)

In [68]:
def convert_to_second(time_str):
    # time_strが文字列の場合にのみ処理
    if isinstance(time_str, str):
        try:
            # "分:秒" の形式で分と秒を分解して、秒に変換
            minutes, seconds = map(float, time_str.split(':'))
            return minutes * 60 + seconds
        except ValueError:
            # 時間のフォーマットが間違っている場合はNoneを返すか、0秒などを返す
            return 0
    # 文字列でない場合（NaNなど）はそのまま返す
    return time_str  
df['Time_x'] = df['Time_x'].apply(convert_to_second)

In [69]:
# Distanceを分離
df['Ground'] = df['Distance'].str[0]
df['Distance'] = df['Distance'].str[1:]

# Ground, Condition, Weatherをダミー変数に変換
dummy = pd.get_dummies(df[['Ground', 'Condition', 'Weather']])

# データフレームを結合し、要らないデータを破棄
df = pd.concat([df, dummy], axis=1)
df.drop(columns=['Ground', 'Condition', 'Weather'], inplace=True)
df

# first_chars = df['Condition'].dropna().apply(lambda x: str(x)[0])  # NaNを除外し、先頭文字を取得
# unique_first_chars = first_chars.unique()  # ユニークな文字を取得

# # 結果を表示
# print("0インデックス目のユニークな文字:", list(unique_first_chars))

,Code,Rank,Frame Rank,Horse Number,Horse Name,Kinryou,Jockey_x,Time_x,Chakusa,Tsuuka,...,Condition_不,Condition_稍,Condition_良,Condition_重,Weather_小雨,Weather_小雪,Weather_晴,Weather_曇,Weather_雨,Weather_雪
0,202436093010,2,1.0,1,ノーブルサターン,59.0,高松亮,101.0,ハナ,3-3-3-3,...,False,False,True,False,False,False,True,False,False,False
1,202436093010,3,8.0,11,マイネルアストリア,54.0,関本玲花,101.5,3,1-1-1-1,...,False,False,True,False,False,False,True,False,False,False
2,202436093010,4,5.0,5,トーセンマッシモ,56.0,塚本涼人,102.0,3,8-10-7-5,...,False,False,True,False,False,False,True,False,False,False
3,202436093010,5,4.0,4,フレイムウィングス,56.0,山本政聡,102.4,2.1/2,8-7-6-7,...,False,False,True,False,False,False,True,False,False,False
4,202436093002,2,8.0,8,ユキノマツシマ,56.0,塚本涼人,51.8,2,3-3,...,False,False,True,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1167665,199010020107,5,5.0,9,コウヒロオリオン,53.0,佐藤哲三,111.6,1.1/2,3-2-3-3,...,False,False,True,False,False,False,False,True,False,False
1167666,199010020111,2,4.0,8,スターオブビジョン,57.0,須貝尚介,121.3,2,4-3-5-6,...,False,False,True,False,False,False,False,True,False,False
1167667,199010020111,3,4.0,7,グリーンフォード,54.0,土肥幸広,121.3,アタマ,4-3-2-2,...,False,False,True,False,False,False,False,True,False,False
1167668,199010020111,4,5.0,10,マリモユーコー,57.0,西園正都,121.4,1/2,7-11-9-6,...,False,False,True,False,False,False,False,True,False,False


In [70]:
# Rankにある"(降)"を除去
df['Rank'] = df['Rank'].astype(str).str.replace(r'[()降]', '', regex = True)
df['Rank'] = df['Rank'].astype(int)

df['Distance'] = df['Distance'].astype(float)

In [71]:
df1 = df[["Rank","Kinryou","Time_x","Nobori","Tansyou","Ninki","Shoukin","Race Number","Distance","Weight","Weight Change","Sex","Age"]]
print('データ型の確認（型変換前）\n{}'.format(df1.dtypes))

データ型の確認（型変換前）
Rank               int64
Kinryou          float64
Time_x           float64
Nobori           float64
Tansyou           object
Ninki            float64
Shoukin           object
Race Number      float64
Distance         float64
Weight             int64
Weight Change      int64
Sex                Int64
Age                Int64
dtype: object


In [72]:
df.to_csv('format_data.csv', index=False)